In [ ]:
pip install djitellopy opencv-python numpy

In [ ]:
# Distance based takeoff

In [ ]:
from djitellopy import Tello
import cv2
import numpy as np
import time

#####################################
# Initialization of coordinates
#####################################
# x points straight, y points left, z points up
# Initial starting position of ego drone (at the center of the picnic blanket)
ego_x = 0
ego_y = 0
ego_z = 0

# State history: stores full state (x, y, z) at each time step
state_history = []  # List of tuples: [(x, y, z), ...]

# Define bounding box around initial starting point (200 x 200)
# Box extends 100 units in each direction from center
BOX_SIZE = 200 # random estimate rn for the bounding box of the ODD, will test based on picnic blanket
BOX_HALF = BOX_SIZE / 2  # 100 units
box_x_min = ego_x - BOX_HALF
box_x_max = ego_x + BOX_HALF
box_y_min = ego_y - BOX_HALF
box_y_max = ego_y + BOX_HALF

#####################################
# Pursuer position (stationary)
#####################################
# Pursuer position (will be set after takeoff to same z-height)
pursuer_x = 100  # Fixed x position of pursuer
pursuer_y = 50   # Fixed y position of pursuer
pursuer_z = None  # Will be set to takeoff height after takeoff

# Threshold distance for collision avoidance (squared distance: x^2 + y^2)
radius = 50
pursuer_threshold = radius**2  # 50^2 = 2500 (50 cm threshold)

#####################################
# Initialization of motion params
#####################################
upward_initialization = 20 #20 cm is the minimum
left_movement  = 50
right_movement = 50
forward_movement = 50
backward_movement = 50

#####################################
# Initialization of drone
#####################################
tello = Tello()
print("Connecting to Tello...")
tello.connect()
print(f"Battery level: {tello.get_battery()}%")
tello.streamon()

print("Press 't' to takeoff, 'q' to land and quit.")
drone_in_air = False

#####################################
# Definitions
#####################################
def compute_switching_filter(new_x, new_y, pursuer_x, pursuer_y):
    dist_to_pursuer_sq = (new_x - pursuer_x)**2 + (new_y - pursuer_y)**2
    boolean_filter = dist_to_pursuer_sq < pursuer_threshold
    return boolean_filter


#####################################
# Drone maneuverance after takeoff
#####################################

try:
    # Record initial state before loop starts
    state_history.append((ego_x, ego_y, ego_z))
    
    while True:
        frame = tello.get_frame_read().frame
        frame = cv2.resize(frame, (720, 480))
        h, w, _ = frame.shape

        # Display the frame
        cv2.imshow("Tello Tracking Feed", frame)

        key = cv2.waitKey(1) & 0xFF

        # Controls for drone
        if key == ord('t') and not drone_in_air:
            tello.takeoff()
            drone_in_air = True
            print("Drone is now airborne and hovering.")
            tello.move_up(upward_initialization)  # move up 30 cm after takeoff
            ego_z += upward_initialization
            pursuer_z = ego_z  # Set pursuer z to same height as takeoff height
            state_history.append((ego_x, ego_y, ego_z))  # Record state after movement
            print(f"Drone moved up 30 cm. Current state: ({ego_x}, {ego_y}, {ego_z})")
            print(f"Pursuer set at position: ({pursuer_x}, {pursuer_y}, {pursuer_z})")
        
        if drone_in_air:
            # WASD key controls for movement
            if key == ord('w'):  # W key - Move forward
                new_x = ego_x + forward_movement
                new_y = ego_y
                # Check distance from pursuer (squared distance to avoid sqrt)
                if compute_switching_filter(new_x, new_y, pursuer_x, pursuer_y):
                    print("CANT MOVE HERE, PURSUER")
                else:
                    print("Moving forward.")
                    ego_x = new_x
                    tello.move_forward(forward_movement)  # Increase movement distance for testing
                    state_history.append((ego_x, ego_y, ego_z))  # Record state after movement
                    print(f"Current state: ({ego_x}, {ego_y}, {ego_z})")
                
            elif key == ord('a'):  # A key - Move left
                new_x = ego_x
                new_y = ego_y + left_movement
                # Check distance from pursuer (squared distance to avoid sqrt)
                if compute_switching_filter(new_x, new_y, pursuer_x, pursuer_y):
                    print("CANT MOVE HERE, PURSUER")
                else:
                    print("Moving left.")
                    ego_y = new_y
                    tello.move_left(left_movement)  # Increase movement distance for testing
                    state_history.append((ego_x, ego_y, ego_z))  # Record state after movement
                    print(f"Current state: ({ego_x}, {ego_y}, {ego_z})")
                
            elif key == ord('s'):  # S key - Move backward
                new_x = ego_x - backward_movement
                new_y = ego_y
                # Check distance from pursuer (squared distance to avoid sqrt)
                if compute_switching_filter(new_x, new_y, pursuer_x, pursuer_y):
                    print("CANT MOVE HERE, PURSUER")
                else:
                    print("Moving backward.")
                    ego_x = new_x
                    tello.move_back(backward_movement)  # Increase movement distance for testing
                    state_history.append((ego_x, ego_y, ego_z))  # Record state after movement
                    print(f"Current state: ({ego_x}, {ego_y}, {ego_z})")
                
            elif key == ord('d'):  # D key - Move right
                new_x = ego_x
                new_y = ego_y - right_movement
                # Check distance from pursuer (squared distance to avoid sqrt)
                if compute_switching_filter(new_x, new_y, pursuer_x, pursuer_y):
                    print("CANT MOVE HERE, PURSUER")
                else:
                    print("Moving right.")
                    ego_y = new_y
                    tello.move_right(right_movement)  # Increase movement distance for testing
                    state_history.append((ego_x, ego_y, ego_z))  # Record state after movement
                    print(f"Current state: ({ego_x}, {ego_y}, {ego_z})")
                
        # Landing control
        if key == ord('q'):
            if drone_in_air:
                tello.land()
                state_history.append((ego_x, ego_y, ego_z))  # Record final state before landing
                print("Drone has landed.")
                print(f"Final state: ({ego_x}, {ego_y}, {ego_z})")
                print(f"Total states recorded: {len(state_history)}")
            break


#####################################
# Landing the drone / finish program
#####################################
except Exception as e:
    print(f"Error occurred: {e}")
    if drone_in_air:
        print("Error occurred. Landing the drone.")
        tello.land()

except KeyboardInterrupt:
    print("Keyboard Interrupt received. Landing the drone.")
    if drone_in_air:
        tello.land()
    tello.streamoff()
    cv2.destroyAllWindows()

finally:
    tello.streamoff()
    cv2.destroyAllWindows()


In [ ]:
# Velocity based takeoff

In [ ]:
from djitellopy import Tello
import cv2
import numpy as np
import time

#####################################
# Initialization of coordinates
#####################################
# x points straight, y points left, z points up
# Initial starting position of ego drone (at the center of the picnic blanket)
ego_x = 0
ego_y = 0
ego_z = 0

# State history: stores full state (x, y, z) at each time step
state_history = []  # List of tuples: [(x, y, z), ...]

# Define bounding box around initial starting point (200 x 200)
# Box extends 100 units in each direction from center
BOX_SIZE = 200 # random estimate rn for the bounding box of the ODD, will test based on picnic blanket
BOX_HALF = BOX_SIZE / 2  # 100 units
box_x_min = ego_x - BOX_HALF
box_x_max = ego_x + BOX_HALF
box_y_min = ego_y - BOX_HALF
box_y_max = ego_y + BOX_HALF

#####################################
# Pursuer position (stationary)
#####################################
# Pursuer position (will be set after takeoff to same z-height)
pursuer_x = 100  # Fixed x position of pursuer
pursuer_y = 50   # Fixed y position of pursuer
pursuer_z = None  # Will be set to takeoff height after takeoff

# Threshold distance for collision avoidance (squared distance: x^2 + y^2)
radius = 50
pursuer_threshold = radius**2  # 50^2 = 2500 (50 cm threshold)

#####################################
# Initialization of motion params
#####################################
upward_initialization = 20 #20 cm is the minimum
left_movement  = 50
right_movement = 50
forward_movement = 50
backward_movement = 50

#####################################
# Initialization of drone
#####################################
tello = Tello()
print("Connecting to Tello...")
tello.connect()
print(f"Battery level: {tello.get_battery()}%")
tello.streamon()

print("Press 't' to takeoff, 'q' to land and quit.")
drone_in_air = False

#####################################
# Definitions
#####################################
def compute_switching_filter(new_x, new_y, pursuer_x, pursuer_y):
    dist_to_pursuer_sq = (new_x - pursuer_x)**2 + (new_y - pursuer_y)**2
    boolean_filter = dist_to_pursuer_sq < pursuer_threshold
    return boolean_filter


#####################################
# Drone maneuverance after takeoff
#####################################

try:
    # Record initial state before loop starts
    state_history.append((ego_x, ego_y, ego_z))
    
    while True:
        frame = tello.get_frame_read().frame
        frame = cv2.resize(frame, (720, 480))
        h, w, _ = frame.shape

        # Display the frame
        cv2.imshow("Tello Tracking Feed", frame)

        key = cv2.waitKey(1) & 0xFF

        # Controls for drone
        # if key == ord('t') and not drone_in_air:
        #     tello.takeoff()
        #     drone_in_air = True
        #     print("Drone is now airborne and hovering.")
        #     tello.move_up(upward_initialization)  # move up 30 cm after takeoff
        #     ego_z += upward_initialization
        #     pursuer_z = ego_z  # Set pursuer z to same height as takeoff height
        #     state_history.append((ego_x, ego_y, ego_z))  # Record state after movement
        #     print(f"Drone moved up 30 cm. Current state: ({ego_x}, {ego_y}, {ego_z})")
        #     print(f"Pursuer set at position: ({pursuer_x}, {pursuer_y}, {pursuer_z})")
        if key == ord('t') and not drone_in_air:
            tello.takeoff()
            drone_in_air = True
            print("Drone is now airborne.")
            
            # Gradual takeoff using velocity control
            target_height = 30  # cm you want to reach
            hover_speed = 10    # cm/s (upward velocity)
            steps = int(target_height / hover_speed)
            
            for _ in range(steps):
                tello.send_rc_control(0, 0, hover_speed, 0)  # (left-right, forward-back, up-down, yaw)
                ego_z += hover_speed
                state_history.append((ego_x, ego_y, ego_z))
                time.sleep(1)  # sleep 1 second per step

            # Stop upward motion and hover
            tello.send_rc_control(0, 0, 0, 0)
            pursuer_z = ego_z
            print(f"Reached hover height: {ego_z} cm")

        
        if drone_in_air:
            # WASD key controls for movement
            if key == ord('w'):  # W key - Move forward
                new_x = ego_x + forward_movement
                new_y = ego_y
                # Check distance from pursuer (squared distance to avoid sqrt)
                if compute_switching_filter(new_x, new_y, pursuer_x, pursuer_y):
                    print("CANT MOVE HERE, PURSUER")
                else:
                    print("Moving forward.")
                    ego_x = new_x
                    tello.move_forward(forward_movement)  # Increase movement distance for testing
                    state_history.append((ego_x, ego_y, ego_z))  # Record state after movement
                    print(f"Current state: ({ego_x}, {ego_y}, {ego_z})")
                
            elif key == ord('a'):  # A key - Move left
                new_x = ego_x
                new_y = ego_y + left_movement
                # Check distance from pursuer (squared distance to avoid sqrt)
                if compute_switching_filter(new_x, new_y, pursuer_x, pursuer_y):
                    print("CANT MOVE HERE, PURSUER")
                else:
                    print("Moving left.")
                    ego_y = new_y
                    tello.move_left(left_movement)  # Increase movement distance for testing
                    state_history.append((ego_x, ego_y, ego_z))  # Record state after movement
                    print(f"Current state: ({ego_x}, {ego_y}, {ego_z})")
                
            elif key == ord('s'):  # S key - Move backward
                new_x = ego_x - backward_movement
                new_y = ego_y
                # Check distance from pursuer (squared distance to avoid sqrt)
                if compute_switching_filter(new_x, new_y, pursuer_x, pursuer_y):
                    print("CANT MOVE HERE, PURSUER")
                else:
                    print("Moving backward.")
                    ego_x = new_x
                    tello.move_back(backward_movement)  # Increase movement distance for testing
                    state_history.append((ego_x, ego_y, ego_z))  # Record state after movement
                    print(f"Current state: ({ego_x}, {ego_y}, {ego_z})")
                
            elif key == ord('d'):  # D key - Move right
                new_x = ego_x
                new_y = ego_y - right_movement
                # Check distance from pursuer (squared distance to avoid sqrt)
                if compute_switching_filter(new_x, new_y, pursuer_x, pursuer_y):
                    print("CANT MOVE HERE, PURSUER")
                else:
                    print("Moving right.")
                    ego_y = new_y
                    tello.move_right(right_movement)  # Increase movement distance for testing
                    state_history.append((ego_x, ego_y, ego_z))  # Record state after movement
                    print(f"Current state: ({ego_x}, {ego_y}, {ego_z})")
                
        # Landing control
        if key == ord('q'):
            if drone_in_air:
                tello.land()
                state_history.append((ego_x, ego_y, ego_z))  # Record final state before landing
                print("Drone has landed.")
                print(f"Final state: ({ego_x}, {ego_y}, {ego_z})")
                print(f"Total states recorded: {len(state_history)}")
            break


#####################################
# Landing the drone / finish program
#####################################
except Exception as e:
    print(f"Error occurred: {e}")
    if drone_in_air:
        print("Error occurred. Landing the drone.")
        tello.land()

except KeyboardInterrupt:
    print("Keyboard Interrupt received. Landing the drone.")
    if drone_in_air:
        tello.land()
    tello.streamoff()
    cv2.destroyAllWindows()

finally:
    tello.streamoff()
    cv2.destroyAllWindows()
